In [ ]:
# Imports
from IOHexperimenter import IOH_function, IOH_logger
import input_reader
import parameters
import datetime
from shutil import make_archive

In [ ]:
# Create a dictionary that contains algorithms and splitpoints / targets
data = input_reader.create_dict()

In [ ]:
class Switch:
    """ Class to implement the switch between algorithms.

    Parameters:
    ---------------
    func : object, callable
        The function to be optimized.

    budget : int
        The budget that is available for function evaluations.

    data : dict
        A dictionary with tau as key, algorithm as value


    Attibutes:
    ---------------
    None

    """

    def __init__(self, func, budget, data):
        self.func = func
        self.budget = budget * func.number_of_variables
        self.data = data

    def __call__(self):
        """ Runs the switch routine. 
        
        Returns:
        -----------
        best_so_far_variables : array
                The best found solution.

        best_so_far_fvaluet: float
               The fitness value for the best found solution.
        """

        # Initialize parameters
        params = parameters.init_params(self.budget)

        # Iterate through dictionary
        for tau, alg in data.items():
            algorithm = alg(self.func)
            # Set algorithm parameters based on parameters object
            algorithm.set_params(params)

            # Define stopping criteria
            func = self.func
            budget = self.budget

            def stopping_criteria():
                return (func.best_so_far_precision) <= tau or (
                    func.evaluations > budget)

            algorithm.set_stopping_criteria(stopping_criteria)

            # Run algorithm and extract parameters
            algorithm.run()
            params = algorithm.get_params(params)
            
            if self.func.evaluations >= budget:
                break

        # Return best point and fitness value found by last run algorithm
        return algorithm.func.best_so_far_variables, algorithm.func.best_so_far_precision

In [ ]:
# Experiment routine
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(
    now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
dimensions = [2]
instances = [1, 2, 3, 4, 5]
func_ids = [15]
exp_budget = 20000
runs = 5
switch_combination = "MLSL-BFGS"
experiment_name = path + "switch-routine/" + switch_combination + "/experiment" + experiment_ID

for func_id in func_ids:
    for dim in dimensions:
        for inst in instances:
            f = IOH_function(func_id, dim, inst, suite='BBOB')
            log = IOH_logger(experiment_name, "run",
                             "switch-routine" + switch_combination, "switch-routine")
            f.add_logger(log)
            for run in range(0, runs):
                if run > 0:
                    f.reset()
                switch = Switch(f, exp_budget, data)
                x_opt, prec = switch()
                print(f' run {run} dim{dim} f{func_id} inst{inst} complete, precision: {prec}')
            f.clear_logger()

make_archive(experiment_name, "zip", experiment_name)